# Train model

## Import thư viện

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
import torch
import transformers
from transformers import BertModel, BertTokenizer
import joblib

## Load data

Load dữ liệu đã được thu thập và chuẩn hoá ở bước Data Visualization vào dataframe.

In [ ]:
file = 'formated__dataset.csv'
df = pd.read_csv(file,delimiter=',', header=None)
print(df.shape)

#Sử dụng 300 record đầu tiên để train model.
df= df.iloc[1:300]
df.head()

## Load pre-train model 

Load pre-train model của BERT từ thư viện.<br>
Tiến hành tokenzier dữ liệu để chuẩn bị cho bước cập nhập trọng số cho model.

In [ ]:
model = BertModel.from_pretrained('bert-base-uncased')
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
#encode lines
tokenized = df[0].apply((lambda x: tokenizer.encode(x, add_special_tokens = True, max_length=512,truncation=True)))
print('encode',tokenized[10])
# decode
print('decode',tokenizer.decode(tokenized[10]))

## Fine-tuning model

Thực hiện cập nhập thay đổi trọng số cho model bằng dữ liệu của chúng ta.

In [ ]:
#get all label 
labels = df[1].to_numpy().astype(np.float)
print('labels shape:', labels.shape)
# get lenght max of tokenized
max_len = 0
for i in tokenized.values:
    if len(i) > max_len:
        max_len = len(i)
print('max len:', max_len)

# if lenght of tokenized not equal max_len , so padding value 0
padded = np.array([i + [0]*(max_len-len(i)) for i in tokenized.values])
print('padded:', padded[1])
print('len padded:', padded.shape)

#get attention mask ( 0: not has word, 1: has word)
attention_mask = np.where(padded ==0, 0,1)
print('attention mask:', attention_mask[1])

# Convert input to tensor
padded = torch.tensor(padded,dtype=torch.long)
attention_mask = torch.tensor(attention_mask)


# Train model
with torch.no_grad():
    last_hidden_states = model(padded, attention_mask =attention_mask)
#     print('last hidden states:', last_hidden_states)

features = last_hidden_states[0][:,0,:].numpy()
print('features:', features)

X_train, X_test, y_train, y_test = train_test_split(features, labels)

cl = LogisticRegression()
cl.fit(X_train, y_train)



## Save model

Đánh giá model và lưu lại model sử dụng cho bước dự đoán.

In [ ]:
# Save model
sc = cl.score(X_test, y_test)
print('score:', sc)
joblib.dump(cl, 'save_model.pkl')
